## File Recursion

In [1]:
import os

class FileRecursion:
    """
    Method to find file with a given suffix
    Arguments:
        suffix   : File extension e.g. .c
        path     : File path to be search for file with given suffix. File path can be a regular file as well
        file_list: Result set returned
    """
    @staticmethod
    def _find_files(suffix, path, file_list):
        # Base case for recursion
        if path is None:
            return
        # If the given path is not a directory but a file
        if os.path.isfile(path):
            # Check if the file ends with required suffix
            if path.endswith(suffix):
                # Add it to the list of results if the file has the suffix
                file_list.append(path)
            else:
                return
        else:
            # If the path is a directory then for every entry in directory recurse
            for content in os.listdir(path):
                FileRecursion._find_files(suffix, os.path.join(path, content), file_list)


    """
    Externally exposed method for finding file(s) with given suffix 
    """
    @staticmethod
    def find_files(suffix, path):
        file_list = []
        FileRecursion._find_files(suffix, path, file_list)
        return file_list
    

## Test Case

In [2]:
import unittest
import mock
import sys

sys.path.append('./src')
import file_recursion

class TestFileRecursion(unittest.TestCase):

    # Path name is a regular file not having the suffix
    @mock.patch('file_recursion.os.path.isfile')
    def test_find_file_when_path_single_file_without_matching_suffix(self, mock_isfile):
        mock_isfile.return_value = True
        self.assertEqual(file_recursion.FileRecursion.find_files('.c', '/users/foo/a.txt'), [])
        self.assertEqual(mock_isfile.call_count, 1)
        mock_isfile.called_with('/users/foo/a.txt')

    # Path name is a regular file having the suffix
    @mock.patch('file_recursion.os.path.isfile')
    def test_find_file_when_path_is_single_file_with_matching_suffix(self, mock_isfile):
        mock_isfile.return_value = True
        self.assertEqual(file_recursion.FileRecursion.find_files('.c', '/users/foo/a.c'), ['/users/foo/a.c'])
        self.assertEqual(mock_isfile.call_count, 1)
        mock_isfile.called_with('/users/foo/a.c')

    # Path name is null
    @mock.patch('file_recursion.os.path.isfile')
    def test_find_file_when_path_is_null(self, mock_isfile):
        self.assertEqual(file_recursion.FileRecursion.find_files('.c', None), [])
        self.assertEqual(mock_isfile.call_count, 0)

    # Path name is a directory with subdirectories
    #
    # /top_dir ----> dir1 -----> a.c
    #    |             |-------> b.c
    # .  |---------> dir2 -----> e.txt
    # top_dir has two directroies 'dir1' & 'dir2'
    # 'dir1' has two matching files - a.c & b.c
    # 'dir2' has a file but suffix of that file doesn't match
    # 
    @mock.patch('file_recursion.os.path.isfile')
    @mock.patch('file_recursion.os.listdir')
    def test_find_file_when_multiple_files_with_suffix_found(self, mock_listdir, mock_isfile):
        mock_isfile.side_effect = (False, False, True, True, False, True)
        mock_listdir.side_effect = (['dir1', 'dir2'], ['a.c', 'b.c'], ['e.txt'])
        self.assertEqual(file_recursion.FileRecursion.find_files('.c', '/top_dir'), ['/top_dir/dir1/a.c', '/top_dir/dir1/b.c'])
        self.assertEqual(mock_isfile.call_count, 6)
        self.assertEqual(mock_listdir.call_count, 3)
        mock_isfile.called_with('/top_dir', '/top_dir/dir1', '/top_dir/dir1/a.c', '/top_dir/dir1/b.c', '/top_dir/dir2', '/top_dir/dir2/e.txt')

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)





....
----------------------------------------------------------------------
Ran 4 tests in 0.005s

OK
